In [1]:
import sys
import os
import numpy as np
import pandas as pd
import datetime
import tqdm
from collections import deque
from math import ceil, sqrt, floor, gamma
from scipy import signal, stats, fftpack 
from sklearn import preprocessing
from PIL import Image
import multiprocessing as mp
import matplotlib.pyplot as plt
# import python library
sys.path.append(os.path.join(os.getcwd().split(os.environ.get('USER'))[0],os.environ.get('USER'), 'wdml', 'py'))

from sample import Sample
from dataset import Dataset
from database import Database
from sample_detector import SampleDetector

dataset_loc = os.path.join(os.getcwd().split(os.environ.get('USER'))[0],os.environ.get('USER'), 'wdml', 'data','datasets', 'awdEvents1')
database_loc = os.path.join(os.getcwd().split(os.environ.get('USER'))[0],os.environ.get('USER'), 'wdml', 'data','databases', 'awdEvents1')
site = 'marion'
my_dataset = Dataset(dataset_loc, site)
my_database = Database(dataset_loc, database_loc, site)
my_sample = SampleDetector(dataset_loc, site, np.random.choice(my_database.get_train(),1)[0])


# my_sample = Detector(dataset_loc, site,'2013-07-04UT15:44:24.35171922.marion.vr2')

my_sample.spectrogram_with_axis(log10=True)
print(my_sample.get_file())
time_err=0.2
diff_err=0.3
freq_size = 3
size = (freq_size*my_sample.get_time_freq_ratio(dec=0), freq_size)

my_sample.spectrogram_plot(figsize=size, zlabel='sprectrum magnitude [dB]', ticks=[20,10],
                           title="Spectrogram: "+my_sample.get_file(), cmap='jet', cmap_show=False)
my_sample.apply_transform(['zscore'],[[None]])
my_sample.spectrogram_plot(figsize=size, zlabel='sprectrum magnitude [dB]', xaxis=True, yaxis=True, ticks=[0.2,2],
                           title="Spectrogram: "+my_sample.get_file(), cmap='jet', cmap_show=False)
kernel = my_sample.whistler_sim(decay=3, whistler_time=0.6, whistler_freq_len=5, whistler_freq_start=2.5, thickness=1)

fig=plt.figure(figsize=(int(size[0]*kernel.shape[1]/my_sample.get_spectrogram().shape[1]),3));img=plt.pcolormesh(kernel,cmap='jet');fig.colorbar(mappable=img, label='Kernel magnitude');plt.title("Whistler Simulation");plt.show();
corr = signal.correlate(10**my_sample.get_spectrogram(), kernel, mode='valid').T
detector = my_sample.apply_detector(corr,'tm_cfar', [10,15,1e-3,0])
# print(my_sample.get_spectrogram().shape,kernel.shape,corr.shape)
# plt.figure(figsize=size);plt.plot(corr);plt.xticks(np.arange(0, len(corr), 20));plt.grid();plt.title("Result of Matched Filter");plt.show()
# plt.figure(figsize=size);plt.plot(np.log10(corr**2));plt.xticks(np.arange(0, len(corr), 20));plt.grid();plt.title("Square of Result of Matched Filter");plt.show()
plt.figure(figsize=size)
plt.plot(np.log10(corr[detector[1]]**2))
plt.plot(np.log10(detector[0]))
plt.xticks(np.arange(0, len(detector[1]), 20));plt.grid();
plt.title('Signal and Detector threshold')
plt.legend(['output of square of matched filter', 'ca-cfar detector threshold'])
plt.show()
plt.figure(figsize=size);plt.plot(my_sample.detection_area(corr,detector));plt.xticks(np.arange(0, len(corr), 20));plt.show()

output_awd = my_sample.awda_whistlers(time_err=time_err)
output_mf = my_sample.awda_whistlers(_output=my_sample.detection_loc(corr,detector,shift=56), time_err=time_err)
print(output_awd)
print(output_mf)
my_sample.spectrogram_with_axis()
print(my_sample.evaluate_detector(['zscore'],[[None]],'tm_cfar', [10,15,1e-3,0], diff_err=diff_err, time_err=time_err))

2013-08-04UT00:02:50.01571914.marion.vr2


<Figure size 1500x300 with 1 Axes>

<Figure size 1500x300 with 1 Axes>

<Figure size 200x300 with 2 Axes>

<Figure size 1500x300 with 1 Axes>

...

<Figure size 1500x300 with 1 Axes>

<Figure size 200x300 with 2 Axes>

<Figure size 1500x300 with 1 Axes>

<Figure size 1500x300 with 1 Axes>

(array([[ 2.43408, 23.     ],
       [ 2.64508, 25.     ],
       [ 2.88508, 29.     ],
       [ 3.23608, 82.     ]]), array([[380,  23],
       [413,  25],
       [451,  29],
       [506,  82]]))
(array([[ 2.513235  , 10.16642491],
       [ 2.717875  , 10.24454089],
       [ 3.255055  , 11.98266107]]), array([[393,  10],
       [425,  10],
       [509,  11]]))
56
(array([[array([ 2.513235  , 10.16642491]), 0],
       [array([ 2.717875  , 10.24454089]), 0],
       [array([ 3.255055  , 11.98266107]), 0],
       [array([ 2.88508, 29.     ]), -1]], dtype=object), array([1, 3, 0]))


In [3]:
import os, sys
from scipy import signal, stats, fftpack 
sys.path.append(os.path.join(os.getcwd().split(os.environ.get('USER'))[0],os.environ.get('USER'), 'wdml', 'py'))
from dataset import Dataset
from database import Database
from detector import Detector
dataset_loc = os.path.join(os.getcwd().split(os.environ.get('USER'))[0],os.environ.get('USER'), 'wdml', 'data','datasets', 'awdEvents1')
database_loc = os.path.join(os.getcwd().split(os.environ.get('USER'))[0],os.environ.get('USER'), 'wdml', 'data','databases', 'awdEvents1')
site = 'marion'
my_dataset = Dataset(dataset_loc, site)
my_database = Database(dataset_loc, database_loc, site)
my_sample = Detector(dataset_loc, site,'2013-05-20UT15:19:54.09342156.marion.vr2')
# my_sample = Detector(dataset_loc, site,'2013-05-18UT14:02:12.39592156.marion.vr2')
# my_sample = Detector(dataset_loc, site,'2013-07-04UT15:48:43.83331922.marion.vr2')
# my_sample.apply_zscore()
# my_sample.apply_transform(['zscore','medfilt'],[[None],[(3,3)]])
my_sample.apply_transform(['zscore'],[[None]])
kernel = my_sample.whistler_sim(decay=3, whistler_time=0.6, whistler_freq_len=5, whistler_freq_start=2.5, thickness=1)
corr = signal.correlate(10**my_sample.get_spectrogram(), kernel, mode='valid').T
# detector = my_sample.tm_cfar(corr,N=10,G=15,C=0,pfa=1e-3)
detector = my_sample.apply_detector(corr, 'tm_cfar', [10,15,1e-3,0])
print(my_sample.output_compare(corr,detector,26,0.3))
print(my_sample.metric(corr,detector,26,0.3))

my_sample.spectrogram_with_axis()
print(my_sample.evaluate_detector(['zscore'],[[None]],'tm_cfar', [10,15,1e-3,0] ))

[[array([ 6.17658, 27.     ]) -1]
 [array([ 5.18814722, 11.16377097]) 0]]
[1 1 0]
(array([[array([ 6.17658, 27.     ]), -1],
       [array([ 5.18814722, 11.16377097]), 0]], dtype=object), array([1, 1, 0]))


In [5]:
# Metric Stats
# # result = []
# # for sample, coor in zip(samples, samples_coor):
# #     my_sample = Sample(dataset_loc, site, sample) 
# #     my_sample.spectrogram_with_axis()
# #     mean_default, std_default = np.round(my_sample.stats(spec=True)[-1],2),np.round(my_sample.snr(coor),2)
# #     my_sample.spectrogram_with_axis()
# #     my_sample.apply_zscore(axis=[1,0])
# #     result.append([np.round(my_sample.stats(spec=True)[-1],2)-mean_default,np.round(my_sample.snr(coor),2)-std_default])
# #     my_sample.spectrogram_with_axis()
# #     my_sample.apply_detrend(axis=[-1,0], type='constant')
# #     result.append([np.round(my_sample.stats(spec=True)[-1],2)-mean_default,np.round(my_sample.snr(coor),2)-std_default])
# #     my_sample.spectrogram_with_axis()
# #     my_sample.apply_detrend(axis=[-1,0], type='linear')
# # #     my_sample.apply_medfilt(kernel=(3,3))
# #     result.append([np.round(my_sample.stats(spec=True)[-1],2)-mean_default,np.round(my_sample.snr(coor),2)-std_default])
# # result = np.array(result)

# result_zscore = []
# for sample, coor in zip(samples, samples_coor):
#     my_sample = Sample(dataset_loc, site, sample) 
#     my_sample.spectrogram_with_axis()
#     mean_default, std_default = np.round(my_sample.stats(spec=True)[-1],2),np.round(my_sample.snr(coor),2)
#     my_sample.spectrogram_with_axis()
# #     my_sample.apply_zscore(axis=[1,0])
# #     my_sample.apply_detrend(axis=[-1,0], type='constant')
#     my_sample.apply_detrend(axis=[-1,0], type='linear')
# #     my_sample.apply_medfilt(kernel=(3,3))
#     my_sample.apply_meanfilt(kernel=(3,3))
#     result_zscore.append([np.round(my_sample.stats(spec=True)[-1],2)-mean_default,np.round(my_sample.snr(coor),2)-std_default])
# result_zscore = np.array(result_zscore)
# print('SPMR: %f+-%f'%(np.round(result_zscore[:,0].mean(),2), np.round(result_zscore[:,0].std(),2)))
# print('SNR: %f+-%f'%(np.round(result_zscore[:,1].mean(),2), np.round(result_zscore[:,1].std(),2)))

# # for ix in range(0,len(result),3):
# #     plt.plot([0,1,2],result[ix:ix+3,0])
# # plt.title('PMS')
# # plt.figure()
# # for ix in range(0,len(result),3):
# #     plt.plot([0,1,2],result[ix:ix+3,1])
# # plt.title('SNR')
# # plt.show()

In [6]:
# dataset_loc = os.path.join(os.getcwd().split(os.environ.get('USER'))[0],os.environ.get('USER'), 'wdml', 'data','datasets', 'awdEvents1')
# database_loc = os.path.join(os.getcwd().split(os.environ.get('USER'))[0],os.environ.get('USER'), 'wdml', 'data','databases', 'awdEvents1')
# site = 'marion'
# my_dataset = Dataset(dataset_loc, site)
# my_database = Database(dataset_loc, database_loc, site)
# # my_sample = Sample(dataset_loc, site, np.random.choice(my_database.get_train(),1)[0])
# samples = ['2013-09-02UT16:17:50.19059414.marion.vr2','2013-07-07UT05:19:49.99651922.marion.vr2','2013-06-28UT04:03:12.85091922.marion.vr2',
#             '2013-10-08UT01:58:59.23379430.marion.vr2','2013-11-09UT22:50:37.90899422.marion.vr2','2013-07-11UT04:36:24.27491938.marion.vr2',
#             '2013-07-25UT04:30:47.73731922.marion.vr2','2013-08-04UT21:34:56.49251922.marion.vr2','2013-02-24UT15:04:07.65347602.marion.vr2',
#             '2013-07-02UT16:10:39.77571914.marion.vr2']
# samples_whistlers_coor = [[[2.2,2.9],[2.9,3.6]],[[2.2,2.9],[5.3,6],[5.8,6.4]],[[2.5,3.2],[4.9,5.5],[6.5,7.2]],
#                           [[2.3,2.6]],[[2.3,2.6],[2.6,3.5]],[[1.2,1.6],[2.3,2.6]],
#                           [[1.8,2.4],[2.4,3.3]],[[2,3]],[[2.3,2.6]],
#                           [[2,3.2],[3.4,3.6],[5.5,6]]]
# samples_whistlers_coor_loc = [[[345,365],[425,455]],[[345,365],[765,855],[865,885]],[[325,395],[725,795],[985,1045]],
#                               [[335,405]],[[45,85],[335,355],[405,445]],[[175,195],[345,355]],
#                               [[265,295],[315,385]],[[325,355]],[[325,375]],
#                               [[285,365],[495,515],[805,855]]]
# samples_whistlers_coor_loc_ = [[[280,520]],[[300,440],[700,960]],[[240,460],[640,880],[940,1100]],
#                               [[260,460]],[[0,140],[240,500]],[[120,240],[280,420]],
#                               [[200,420]],[[180,460]],[[240,420]],
#                               [[220,440],[780,900]]]

# noise, signal_and_noise = np.array([[0]]),np.array([[0]])
# for ix in range(len(samples)):
# #     print(samples[ix])
#     my_sample = Sample(dataset_loc, site, samples[ix])
#     my_sample.spectrogram_with_axis()
#     my_sample.apply_zscore()
#     kernel = my_sample.whistler_sim(decay=4, whistler_time=0.8, whistler_freq_len=7, whistler_freq_start=2.5, thickness=1)
#     corr = signal.correlate(10**my_sample.get_spectrogram(), kernel, mode='valid').T
# #     plt.figure(figsize=(15,3));plt.plot(corr);plt.xticks(np.arange(0, len(corr), 20));plt.show()
#     i = 0
#     for c in samples_whistlers_coor_loc[ix]:
#         noise = np.concatenate((noise,corr[i:c[0]]))
#         signal_and_noise = np.concatenate((signal_and_noise,corr[c[0]:c[1]]))
#         i = c[1]
#     noise = np.concatenate((noise,corr[i:]))
    
# noise, signal_and_noise = np.array(noise), np.array(signal_and_noise)
# print(noise.max(), signal_and_noise.max())
# plt.figure()
# arr = np.histogram(noise, bins=np.linspace(0,1.3e6,100), density=True)
# # print(arr)
# plt.plot(arr[1][:-1],arr[0])
# # plt.plot(np.histogram(noise, bins=10, density=True)[0])
# plt.show()
# plt.figure()
# arr = np.histogram(signal_and_noise, bins=np.linspace(0,28e6,100), density=True)
# plt.plot(arr[1][:-1],arr[0])
# plt.show()